In [133]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 

In [134]:
fraud= pd.read_csv("Fraud_check.csv")

In [108]:
fraud

,Undergrad,MaritalStatus,CityPopulation,WorkExperience,Urban,TaxableIncome
0,NO,Single,50047,10,YES,68833
1,YES,Divorced,134075,18,YES,33700
2,NO,Married,160205,30,YES,36925
3,YES,Single,193264,15,YES,50190
4,NO,Married,27533,28,NO,81002
...,...,...,...,...,...,...
595,YES,Divorced,39492,7,YES,76340
596,YES,Divorced,55369,2,YES,69967
597,NO,Divorced,154058,0,YES,47334
598,YES,Married,180083,17,NO,98592


In [109]:
fraud['TaxableIncome'] = pd.cut(fraud.TaxableIncome,bins=(0,30000,199778),labels=['Risky','Good'])
fraud

,Undergrad,MaritalStatus,CityPopulation,WorkExperience,Urban,TaxableIncome
0,NO,Single,50047,10,YES,Good
1,YES,Divorced,134075,18,YES,Good
2,NO,Married,160205,30,YES,Good
3,YES,Single,193264,15,YES,Good
4,NO,Married,27533,28,NO,Good
...,...,...,...,...,...,...
595,YES,Divorced,39492,7,YES,Good
596,YES,Divorced,55369,2,YES,Good
597,NO,Divorced,154058,0,YES,Good
598,YES,Married,180083,17,NO,Good


In [110]:
fraud.isnull().sum()

Undergrad         0
MaritalStatus     0
CityPopulation    0
WorkExperience    0
Urban             0
TaxableIncome     0
dtype: int64

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.tree import  DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder

In [112]:
x= fraud.iloc[:,:-1]

In [113]:
x

,Undergrad,MaritalStatus,CityPopulation,WorkExperience,Urban
0,NO,Single,50047,10,YES
1,YES,Divorced,134075,18,YES
2,NO,Married,160205,30,YES
3,YES,Single,193264,15,YES
4,NO,Married,27533,28,NO
...,...,...,...,...,...
595,YES,Divorced,39492,7,YES
596,YES,Divorced,55369,2,YES
597,NO,Divorced,154058,0,YES
598,YES,Married,180083,17,NO


In [114]:
y = fraud.iloc[:,5]
y

0      Good
1      Good
2      Good
3      Good
4      Good
       ... 
595    Good
596    Good
597    Good
598    Good
599    Good
Name: TaxableIncome, Length: 600, dtype: category
Categories (2, object): [Risky < Good]

In [115]:
labelencoder_x=LabelEncoder()

In [116]:
x=x.apply(LabelEncoder().fit_transform)

In [117]:
x

,Undergrad,MaritalStatus,CityPopulation,WorkExperience,Urban
0,0,2,84,10,1
1,1,0,398,18,1
2,0,1,481,30,1
3,1,2,574,15,1
4,0,1,4,28,0
...,...,...,...,...,...
595,1,0,55,7,1
596,1,0,107,2,1
597,0,0,459,0,1
598,1,1,533,17,0


In [118]:
## Bagged Decision Trees for Classification

In [119]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [120]:
seed=7
kfold = KFold(n_splits=10, random_state=seed)

C:\Users\SAI HARI\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [121]:
cart = DecisionTreeClassifier()
num_trees = 50

In [122]:
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)

In [123]:
results = cross_val_score(model, x, y, cv=kfold)

In [124]:
print(results.mean())

0.7383333333333333


In [125]:
# Random Forest Classification

In [126]:
num_trees = 100
max_features = 3
kfold = KFold(n_splits=10, random_state=7)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, x, y, cv=kfold)
print(results.mean())

C:\Users\SAI HARI\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0.7383333333333333


In [127]:
# AdaBoost Classification

In [128]:
from sklearn.ensemble import AdaBoostClassifier

In [129]:
num_trees = 10
seed=7
kfold = KFold(n_splits=10, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = cross_val_score(model, x, y, cv=kfold)
print(results.mean())

C:\Users\SAI HARI\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0.7916666666666666


In [130]:
# Stacking Ensemble for Classification

In [131]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [132]:
# create the sub models
estimators = []
model1 = LogisticRegression(max_iter=500)
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))

# create the ensemble model
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble, x, y, cv=kfold)
print(results.mean())

0.7933333333333332
